In [1]:
# headless web browsing 
# testing

using WebDriver, Gumbo, Cascadia
capabilities = Capabilities("safari")
wd = RemoteWebDriver(capabilities, host = "172.30.1.35", port = 4444)
status(wd)
session = Session(wd)
# current_url(session)



Session


In [3]:
# todo
# 
page = 0
cards = nothing
item_count = 0
restaurant_names = String[]
restaurant_ratings = Float64[]
while true
    page += 1
    println(page)
    navigate!(session, "https://www.opentable.com/new-york-restaurant-listings?page=$page")
    for i = 1:35
        script!(session, "window.scrollBy({top: window.innerHeight, left: 0, behavior: 'smooth'});")
        sleep(1)
    end
    html = parsehtml(source(session))
    body_element = html.root[2]
    for e in body_element.children
        try
            if tag(e) == :div
                if getattr(e, "id") == "wrapper"
                    # println("1", e)
                    a = e[1][1]
                    for e2 in a.children
                        if tag(e2) == :main
                            for e3 in e2.children
                                if tag(e3) == :div
                                    b = e3
                                    # println("3", e3)
                                    # print(b)
                                    for e4 in b.children
                                        if tag(e4) == :div
                                            # println("4", e4)
                                            for e5 in e4.children
                                                # println(e5)
                                                if tag(e5) == :div
                                                    
                                                    # println(e5)
                                                    if hasattr(e5, "data-test")
                                                        # println()
                                                        if getattr(e5, "data-test") == "restaurant-cards"
                                                            cards = e5
                                                        end
                                                    end
                                                end
                                            end
                                        end 
                                    end
                                end
                            end
                        end
                    end
                end
            end
        catch
            # Nothing
        end
    end
    for (index, card) in enumerate(children(cards))
        item = card[2][1] 
        name = text(item[1][1][1])
        if length(item.children) == 1
            rating = 0
        else
            mitem = item[2][1]
            if length(mitem.children) == 1
                rating = 0
            else
                rating = parse(Float64, text(mitem[2]))
            end
        push!(restaurant_names, name)
        push!(restaurant_ratings, rating)
        end
    end
    if page == 2
        break
    end
    if length(children(cards)) < 50
        break
    end
end


1
2


In [5]:
using DataFrames
ny_restaurants = DataFrame(name = restaurant_names, rating = restaurant_ratings)

Row,name,rating
,Any,Any
1,Bustan,4.7
2,Elea,4.7
3,Nice Matin,4.5
4,Dagon,4.5
5,Fogo de Chao - New York,4.4
6,Carmine's - 91st Street - NYC,4.5
7,Arco Cafe,4.8
8,Tavern on the Green,4.6
9,5 Napkin Burger - Upper West Side,4.6
